In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Offline Hindi Voice Assistant
Optimized for Raspberry Pi 4 (4GB RAM)
CPU-only | Quantized ASR | Rule-based Intent | eSpeak TTS
"""

# ==========================================================
# STANDARD LIBRARIES
# ==========================================================
import os
import sys
import time
import platform
import datetime
import multiprocessing as mp
import threading
import queue
import subprocess
import psutil
import shutil
import warnings
import ctypes

# ==========================================================
# --- COMPULSORY LIBS CHECK ---
# ==========================================================
try:
    import RPi.GPIO as GPIO
    GPIO_AVAILABLE = True
    GPIO_FAN = 17
    GPIO_LIGHT = 27
except ImportError:
    GPIO_AVAILABLE = False
    if not os.environ.get("GPIO_WARNED"):
        print("Warning: RPi.GPIO not found. GPIO tasks will be simulated.")
        os.environ["GPIO_WARNED"] = "1"


# ==========================================================
# SCIENTIFIC / ML LIBRARIES
# ==========================================================
try:
    import torch
except ImportError:
    print("Torch not installed.")

try:
    from transformers import Wav2Vec2Processor, AutoModelForCTC, AutoTokenizer, AutoModelForSeq2SeqLM ,MarianMTModel, MarianTokenizer
except ImportError:
    print("Transformer not installed.")

try:
    import numpy as np
except ImportError:
    print("NumPy not installed.")
    # #

try:
    import joblib
except ImportError:
    print("Joblib not installed.")
    # #

try:
    import onnxruntime as ort
except ImportError:
    print("ONNX Runtime not installed.")
    # #

try:
    from sklearn.pipeline import Pipeline
except ImportError:
    print("Scikit-learn not installed.")
    # #

# ==========================================================
# AUDIO LIBRARIES
# ==========================================================
try:
    import pyaudio
except ImportError:
    print("PyAudio not installed.")
    # #

try:
    import webrtcvad
except ImportError:
    print("WebRTC VAD not installed.")
    # #

# ==========================================================
# CONFIGURATION
# ==========================================================

INTENT_MODEL_PATH = "models/intent_model_optimized.joblib"

SAMPLE_RATE = 16000
CHANNELS = 1
CHUNK = 320  # 20 ms frames at 16kHz
VAD_MODE = 2

# ==========================================================
# SYSTEM CHECKS
# ==========================================================

def check_system():
    print("\n=== SYSTEM CHECK ===")

    print("Platform:", platform.platform())
    print("Python Version:", platform.python_version())
    print("CPU Cores:", psutil.cpu_count(logical=False))
    print("Logical Cores:", psutil.cpu_count())
    print("Total RAM (GB):", round(psutil.virtual_memory().total / (1024**3), 2))

    if psutil.virtual_memory().total < 3 * (1024**3):
        print("Warning: RAM may be insufficient.")

    print("System check passed.\n")


# ==========================================================
# CPU OPTIMIZATION
# ==========================================================

def optimize_cpu():
    print("Optimizing CPU usage...")

    # Limit OpenMP threads for ONNX Runtime
    os.environ["OMP_NUM_THREADS"] = "2"
    os.environ["MKL_NUM_THREADS"] = "2"

    # Pin main process to specific cores (0,1,2)
    try:
        p = psutil.Process(os.getpid())
        p.cpu_affinity([0, 1, 2])
        print("CPU affinity set.")
    except Exception as e:
        print("CPU affinity not supported:", e)

    print("CPU optimization complete.\n")


# ==========================================================
# MODEL CHECK
# ==========================================================

def check_intent_model():
    print("Checking models...")

    try :
        os.path.exists(INTENT_MODEL_PATH)
        print(f"Intent model found: {INTENT_MODEL_PATH}")
    except Exception as e:
        print("Model Not found")
        # #

    print("Model files verified.\n")


# ==========================================================
# REQUIRED MODELS CONFIGURATION
# ==========================================================

ASR_MODELS = {
    "coqui_stt": "models\ASR_STT\coqui_stt\coqui_stt_asr_model.tflite",
    "wav2vec2": "models\ASR_STT\IndicWav2Vec2\pytorch_model.bin",
    # "wav2vec2": "models\ASR_STT\wav2vec2\model.safetensors",
    "vosk": "models/ASR_STT/vosk-model-small-hi-0.22/vosk-model-small-hi-0.22",
    "whisper": "models\ASR_STT\Wisper-small\model.safetensors"
}

ASR_model_select="coqui_stt"

# System binaries
SYSTEM_TTS_ENGINES = {
    "espeak-ng": "models/espeak-ng",
    "festival": "models/festival"
}

TTS_model_select="espeak-ng"

TRANSLATION_MODEL={
    "hi-en_model": "models\Translation_model\hi-en_model"
}

TRANSLATION_model_select="hi-en_model"

# ==========================================================
# FILE SIZE UTILITY
# ==========================================================

def get_size_mb(path):
    if os.path.isfile(path):
        return round(os.path.getsize(path) / (1024**2), 2)
    elif os.path.isdir(path):
        total = 0
        for root, _, files in os.walk(path):
            for f in files:
                total += os.path.getsize(os.path.join(root, f))
        return round(total / (1024**2), 2)
    return 0


# ==========================================================
# ASR MODEL CHECK
# ==========================================================

def check_asr_models():
    print("Checking ASR models...\n")

    for name, path in ASR_MODELS.items():
        if not os.path.exists(path):
            print(f"[MISSING] {name} → {path}")
        else:
            size = get_size_mb(path)
            print(f"[OK] {name} found | Size: {size} MB")

    print("\nASR model check complete.\n")


# ==========================================================
# TTS MODEL CHECK
# ==========================================================

def check_tts_models():
    print("Checking TTS models...\n")

    # System binaries check
    for name, cmd in SYSTEM_TTS_ENGINES.items():
        if shutil.which(cmd) is None:
            print(f"[MISSING] System TTS Engine: {name}")
        else:
            print(f"[OK] {name} installed")

    print("\nTTS model check complete.\n")


# ==========================================================
# LIBRARY CHECK FOR ASR ENGINES
# ==========================================================

def check_asr_libraries():
    print("Checking ASR libraries...\n")

    try:
        import stt
        print("[OK] Coqui STT library installed")
    except ImportError:
        print("[MISSING] Coqui STT library")

    try:
        import vosk 
        from vosk import Model, KaldiRecognizer
        print("[OK] Vosk library installed")
    except ImportError:
        print("[MISSING] Vosk library")

    try:
        import whisper
        print("[OK] Whisper library installed")
    except ImportError:
        print("[MISSING] Whisper library")

    print("\nASR library check complete.\n")

# ==========================================================
# TRANSLATION MODEL CHECK
# ==========================================================

def check_translation_models():
    print("Checking translation models...\n") 
    for name, path in TRANSLATION_MODEL.items():
        if not os.path.exists(path):
            print(f"[MISSING] {name} → {path}")
        else:
            size = get_size_mb(path)
            print(f"[OK] {name} found | Size: {size} MB")

    print("\nTranslation model check complete.\n")



# ==========================================================
# LIBRARY CHECK FOR ASR ENGINES
# ==========================================================

def check_tts_libraries():
    print("Checking TTS libraries...\n")
    try:
        import sounddevice
        print("[OK] sounddevice library installed")
    except ImportError:
        print("[WARNING] sounddevice not installed (audio playback may fail)")

    try:
        import pyttsx3
        print("[OK] pyttsx3 installed (fallback TTS available)")
    except ImportError:
        print("[INFO] pyttsx3 not installed (optional)")

    print("\nTTS library check complete.\n")


# ==========================================================
# AUDIO DEVICE CHECK
# ==========================================================

def check_audio_devices():
    print("Checking audio devices...")

    pa = pyaudio.PyAudio()

    input_found = False
    output_found = False

    for i in range(pa.get_device_count()):
        dev = pa.get_device_info_by_index(i)
        if dev["maxInputChannels"] > 0:
            input_found = True
        if dev["maxOutputChannels"] > 0:
            output_found = True

    pa.terminate()

    if not input_found:
        print("No microphone detected.")
        # #

    if not output_found:
        print("No speaker detected.")
        # #

    print("Audio devices OK.\n")


# ==========================================================
# ASR MODEL PRELOAD 
# ==========================================================

def preload_asr_model():
    print("Loading ASR model...")
    asr_model_path = ASR_MODELS.get(ASR_model_select)
    
    if ASR_model_select == "coqui_stt":
        # Load Coqui STT (TensorFlow Lite)
        try:
            import stt
            session = stt.Model(asr_model_path)
            print("Coqui STT model loaded.")
        except Exception as e:
            print(f"Error loading Coqui STT model: {e}")
            return None

    elif ASR_model_select == "wav2vec2":
        # Load Wav2Vec2 (PyTorch-based)
        try:
            model = torch.load(asr_model_path)
            print("Wav2Vec2 model loaded.")
            return model
        except Exception as e:
            print(f"Error loading Wav2Vec2 model: {e}")
            return None

    elif ASR_model_select == "vosk":
        # Load Vosk (Kaldi-based)
        try:
            model = Model(asr_model_path)
            recognizer = KaldiRecognizer(model, SAMPLE_RATE)
            print("Vosk model loaded.")
            return recognizer
        except Exception as e:
            print(f"Error loading Vosk model: {e}")
            return None

    elif ASR_model_select == "whisper":
        # Load Whisper (OpenAI)
        try:
            import whisper
            model = whisper.load_model(asr_model_path)
            print("Whisper model loaded.")
            return model
        except Exception as e:
            print(f"Error loading Whisper model: {e}")
            return None

    return None


# ==========================================================
# TTS MODELS PRELOAD
# ==========================================================
def preload_tts_model():
    print("Loading TTS model...")

    if TTS_model_select == "espeak-ng":
        try:
            # Check if espeak-ng is installed and available
            if shutil.which("espeak-ng") is not None:
                print("eSpeak-NG TTS model loaded.")
                return "espeak-ng"
            else:
                print("[MISSING] eSpeak-NG TTS engine not found.")
                return None
        except Exception as e:
            print(f"Error loading eSpeak-NG: {e}")
            return None

    elif TTS_model_select == "festival":
        try:
            # Check if Festival is installed and available
            if shutil.which("festival") is not None:
                print("Festival TTS model loaded.")
                return "festival"
            else:
                print("[MISSING] Festival TTS engine not found.")
                return None
        except Exception as e:
            print(f"Error loading Festival: {e}")
            return None

    return None


# ==========================================================
# INTENT MODEL PRELOAD
# ==========================================================

def preload_intent_model():
    print("Loading intent model...")
    model = joblib.load(INTENT_MODEL_PATH)
    print("Intent model loaded.\n")
    return model

# ==========================================================
# TRANSLATION MODEL PRELOAD
# ==========================================================

def preload_translation_model():
    print("Loading translation model...")
    translation_model_path = TRANSLATION_MODEL.get(TRANSLATION_model_select)
    tokenizer = MarianTokenizer.from_pretrained(translation_model_path)
    translator = MarianMTModel.from_pretrained(translation_model_path)
    translator.eval()
    print("Hindi → English Translator model loaded. \n")
    return tokenizer,translator

# ==========================================================
# MAIN INITIALIZATION
# ==========================================================

def initialize():
    warnings.filterwarnings("ignore")

    check_system()
    optimize_cpu()
    check_asr_models()
    check_tts_models()
    check_translation_models()
    check_asr_libraries()
    check_tts_libraries()
    check_intent_model()
    check_audio_devices()

    asr_session = preload_asr_model()
    intent_model = preload_intent_model()
    tts_model = preload_tts_model()
    tokenizer,translator =preload_translation_model()
    print("All model checks completed.\n")
    return asr_session, intent_model


# ==========================================================
# ENTRY POINT
# ==========================================================

if __name__ == "__main__":
    mp.set_start_method("spawn", force=True)

    print("\n=== Starting Offline Hindi Voice Assistant Dependencies Check ===\n")

    asr_session, intent_model = initialize()

    print("Initialization complete.")
    print("System ready.\n")

    # Pipeline execution will start here (next stage)


WebRTC VAD not installed.

=== Starting Offline Hindi Voice Assistant Dependencies Check ===


=== SYSTEM CHECK ===
Platform: Windows-10-10.0.26200-SP0
Python Version: 3.10.11
CPU Cores: 8
Logical Cores: 16
Total RAM (GB): 15.4
System check passed.

Optimizing CPU usage...
CPU affinity set.
CPU optimization complete.

Checking ASR models...

[OK] coqui_stt found | Size: 45.22 MB
[OK] wav2vec2 found | Size: 1203.71 MB
[OK] vosk found | Size: 78.28 MB
[OK] whisper found | Size: 922.2 MB

ASR model check complete.

Checking TTS models...

[MISSING] System TTS Engine: espeak-ng
[MISSING] System TTS Engine: festival

TTS model check complete.

Checking translation models...

[OK] hi-en_model found | Size: 293.8 MB

Translation model check complete.

Checking ASR libraries...

[MISSING] Coqui STT library
[OK] Vosk library installed
[MISSING] Whisper library

ASR library check complete.

Checking TTS libraries...

[OK] sounddevice library installed
[OK] pyttsx3 installed (fallback TTS availab